In [5]:
### CODE GENERATED USING CHATGPT
import os
import re
import pandas as pd

# --- Config ---
metrics_dir = 'onedrive_csvs' #change to path to directory containing CSV files
patches_csv = 'patches.csv' #CSV containing names of files in Google Drive patch directory
annotations_csv = 'annotations.csv' #CSV containing names of files in OneDrive annotation directory
patched_csv = 'patched.csv' #CSV containing names of files in OneDrive patched image directory
output_excel = 'patch_summary.xlsx' #change to name you want for output file

# --- Helper Functions ---
#normalizes slice names to account for leading 0s
def normalize_slice_id(raw_id): 
    match = re.match(r"case_(\d+)_(un)?match_(\d+)_(.+)", raw_id)
    if match:
        case_num = int(match.group(1))              # Removes leading zero
        match_type = "unmatched" if match.group(2) == "un" else "match"
        match_num = int(match.group(3))             # Removes leading zero
        stain = match.group(4)
        return f"case_{case_num}_{match_type}_{match_num}_{stain}"
    return raw_id

#splits filenames into slice, match, stain
def parse_slice_components(normalized_id):
    match = re.match(r"case_(\d+)_(match|unmatched)_(\d+)_(.+)", normalized_id)
    if match:
        case_number = int(match.group(1))
        match_type = match.group(2)
        match_number = int(match.group(3))
        stain = match.group(4)
        return case_number, match_type, match_number, stain
    return None, None, None, None


# --- Step 1: Expected patch counts from metrics_*.csv ---

expected_patch_counts = {}

for filename in os.listdir(metrics_dir):
    if filename.startswith("metrics_") and filename.endswith(".csv"):
        filepath = os.path.join(metrics_dir, filename)
        df = pd.read_csv(filepath)
        slice_id = filename.replace("metrics_", "").replace(".csv", "")
        num_patches = df.set_index("Metric").loc["num_patches", "Value"]
        expected_patch_counts[slice_id] = num_patches

# --- Step 2: Actual patch counts from patches.csv ---

patch_df = pd.read_csv(patches_csv, header=None, names=["filename"])
patch_df["slice_id"] = patch_df["filename"].str.extract(r"^(.*)_patch\d+\.png$")
patch_df = patch_df.dropna(subset=["slice_id"])
patch_counts = patch_df["slice_id"].value_counts().to_dict()

# --- Step 3: Slice-level summary ---

all_slices = set(expected_patch_counts) | set(patch_counts)
summary_data = []

for slice_id in sorted(all_slices):
    expected = expected_patch_counts.get(slice_id, 0)
    actual = patch_counts.get(slice_id, 0)
    normalized_id = normalize_slice_id(slice_id)
    summary_data.append({
        "slice_id": slice_id,
        "normalized_slice_id": normalized_id,
        "expected_num_patches": expected,
        "actual_num_patches": actual,
        "difference": actual - expected
    })

summary_df = pd.DataFrame(summary_data)

# --- Step 4: Load annotation and patched image filenames ---

annotations_df = pd.read_csv(annotations_csv, header=None, names=["filename"])
annotations_df["slice_id"] = annotations_df["filename"].str.replace(".png", "", regex=False)
annotations_df["normalized_slice_id"] = annotations_df["slice_id"].apply(normalize_slice_id)

patched_df = pd.read_csv(patched_csv, header=None, names=["filename"])
patched_clean = patched_df[
    patched_df["filename"].str.startswith("patched_") &
    ~patched_df["filename"].str.contains("_patch")
].copy()
patched_clean["slice_id"] = (
    patched_clean["filename"]
    .str.replace("patched_", "", regex=False)
    .str.replace(".png", "", regex=False)
)
patched_clean["normalized_slice_id"] = patched_clean["slice_id"].apply(normalize_slice_id)

# --- Step 5: Add annotation and patched flags ---

summary_df["has_annotation"] = summary_df["normalized_slice_id"].isin(annotations_df["normalized_slice_id"])
summary_df["has_patched_image"] = summary_df["normalized_slice_id"].isin(patched_clean["normalized_slice_id"])

# --- Step 6: Parse identifiers into components ---

parsed = summary_df["normalized_slice_id"].apply(parse_slice_components)
summary_df[["case_number", "match_type", "match_number", "stain"]] = pd.DataFrame(parsed.tolist(), index=summary_df.index)

# --- Step 7: Count slices per case_number (only actual > 0) ---

slices_per_case = (
    summary_df[summary_df["actual_num_patches"] > 0]
    .groupby("case_number")
    .size()
    .reset_index(name="num_slices_patched")
)

# --- Step 8: Export to Excel ---

with pd.ExcelWriter(output_excel, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="summary", index=False)
    slices_per_case.to_excel(writer, sheet_name="slices_per_case", index=False)

print(f"✅ Excel summary written to: {output_excel}")


✅ Excel summary written to: patch_summary.xlsx


In [9]:
import pandas as pd
import re

# Load files
summary_df = pd.read_excel("patch_summary.xlsx", sheet_name="summary") # the output from cell above
slice_patches_df = pd.read_csv("case_slices.csv") # CSV of "Slices per Case (B/H)" from Krish's Patching slices spreadsheet

# ---- Normalize case identifiers ----
def normalize_case_id(val):
    match = re.match(r"^0*(\d+)$", str(val))
    return int(match.group(1)) if match else val

summary_df["normalized_case_number"] = summary_df["case_number"].apply(normalize_case_id)
slice_patches_df["Case"] = slice_patches_df["Case"].apply(normalize_case_id)

# ---- Determine completion status ----
def determine_status(row):
    has_annotation = row["has_annotation"]
    has_patched_image = row["has_patched_image"]
    expected = row["expected_num_patches"]
    actual = row["actual_num_patches"]
    
    if actual == 0 and not has_annotation:
        return "incomplete"
    elif expected == actual and has_annotation and has_patched_image:
        return "complete"
    else:
        return "partial"


summary_df["completion_status"] = summary_df.apply(determine_status, axis=1)

# ---- Merge slice data with case data ----
case_summary = pd.merge(
    slice_patches_df,
    summary_df.groupby("normalized_case_number")["completion_status"]
        .value_counts()
        .unstack(fill_value=0)
        .reset_index(),
    left_on="Case",
    right_on="normalized_case_number",
    how="left"
)

# Optional: compute total processed slices
case_summary["slices_found_in_outputs"] = case_summary[["complete", "partial", "incomplete"]].sum(axis=1)

# ---- Save results ----
with pd.ExcelWriter("project_overview.xlsx") as writer:
    summary_df.to_excel(writer, sheet_name="Per Slice Summary", index=False)
    case_summary.to_excel(writer, sheet_name="Case Summary", index=False)

print("✅ project_overview.xlsx generated successfully.")


✅ project_overview.xlsx generated successfully.


In [10]:
#updates spreadsheet from cell above to check against expected number of slices per case

# ---- Step 1: Generate slice counts per case from summary ----
slice_counts_by_case = (
    summary_df.groupby("normalized_case_number")["completion_status"]
    .value_counts()
    .unstack(fill_value=0)
    .rename_axis(index="Case")
    .reset_index()
)

# ---- Step 2: Merge into slice_patches_df ----
slice_patches_merged = pd.merge(
    slice_patches_df,
    slice_counts_by_case,
    on="Case",
    how="left"
)

# Fill in 0s for any missing complete/partial/incomplete columns
for col in ["complete", "partial", "incomplete"]:
    if col not in slice_patches_merged:
        slice_patches_merged[col] = 0

# ---- Step 3: Calculate summary columns ----
slice_patches_merged["actual_total_slices_found"] = (
    slice_patches_merged["complete"] +
    slice_patches_merged["partial"] +
    slice_patches_merged["incomplete"]
)

slice_patches_merged["slices_missing_from_output"] = (
    slice_patches_merged["Total slices"] - slice_patches_merged["actual_total_slices_found"]
)

# ---- Step 4: Save updated Excel ----
with pd.ExcelWriter("project_overview.xlsx") as writer:
    summary_df.to_excel(writer, sheet_name="Per Slice Summary", index=False)
    slice_patches_merged.to_excel(writer, sheet_name="Expanded Case Summary", index=False)
    print("✅ project_overview.xlsx updated with expanded summary.")


✅ project_overview.xlsx updated with expanded summary.
